In [15]:
import tensorflow as tf
import numpy as np
sess = tf.InteractiveSession()

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [4]:
n_inputs = 28*28
n_hidden1 = 150
n_hidden2 = 150
n_hidden3 = 150
n_outputs = 10

In [5]:
reset_graph()
X = tf.placeholder(tf.float32, shape = (None, n_inputs), name = "X")
y = tf.placeholder(tf.int64, shape = (None), name = "y")

In [6]:
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    hidden3 = tf.layers.dense(hidden2, n_hidden3, name="hidden3", activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

In [7]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy,name="loss")

In [8]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [9]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y , 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [10]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [38]:
n_epochs = 30
n_batches = 50
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict = {X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict = {X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
        print(epoch, "Train accuracy:", acc_train, "test accuracy:", acc_test)
    %%timeit -n1 -r5 accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Train accuracy: 0.92 test accuracy: 0.8934
1 Train accuracy: 0.86 test accuracy: 0.9133
2 Train accuracy: 0.86 test accuracy: 0.9256
3 Train accuracy: 0.96 test accuracy: 0.9313
4 Train accuracy: 0.96 test accuracy: 0.9367
5 Train accuracy: 0.96 test accuracy: 0.9415
6 Train accuracy: 0.98 test accuracy: 0.9445
7 Train accuracy: 0.96 test accuracy: 0.947
8 Train accuracy: 0.98 test accuracy: 0.9504
9 Train accuracy: 1.0 test accuracy: 0.9539
10 Train accuracy: 1.0 test accuracy: 0.955
11 Train accuracy: 1.0 test accuracy: 0.9589
12 Train accuracy: 1.0 test accuracy: 0.9595
13 Train accuracy: 1.0 test accuracy: 0.9605
14 Train accuracy: 0.98 test accuracy: 0.9623
15 Train accuracy: 1.0 test accuracy: 0.9638
16 Train accuracy: 0.96 test accuracy: 0.9645
17 Train accuracy: 0.98 test accuracy: 0.9659
18 Train accuracy: 1.0 test accuracy: 0.9672
19 Train accuracy: 0.96 test accuracy: 0.9687
20 Train accuracy: 0.96 test accuracy: 0.9681
21 Train accuracy: 0.98 test accuracy: 0.969
22 Train

In [27]:
mnist.test.labels.shape

(10000,)